# Sklearn Pipeline
<img src="pipeline-diagram.png">
O pipeline tem como objetivo automatizar uma sequência de comandos.  
Exemplo: usando pipeline para fazer a seleção dos parâmetros de pré processamento e dos hyperparâmetros do modelo?

Vantagens do CV pipeline: one-hot encoder transform dentro do pipeline com a nova versão do sklearn (0.20.2 ou + recente).  
  

Usando dados do Titanic, descritos em: https://www.openml.org/d/40945

## Análise exploratória dos dados

In [ ]:
import pandas as pd
from sklearn.metrics import plot_confusion_matrix
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt 

In [ ]:
#df = pd.read_csv('http://bit.ly/kaggletrain')
df = pd.read_csv('https://www.openml.org/data/get_csv/16826755/phpMYEkMl', na_values='?')

In [ ]:
df.shape

(1309, 14)

In [ ]:
df.columns

Index(['pclass', 'survived', 'name', 'sex', 'age', 'sibsp', 'parch', 'ticket',
       'fare', 'cabin', 'embarked', 'boat', 'body', 'home.dest'],
      dtype='object')

In [ ]:
df.cabin

0            B5
1       C22 C26
2       C22 C26
3       C22 C26
4       C22 C26
         ...   
1304        NaN
1305        NaN
1306        NaN
1307        NaN
1308        NaN
Name: cabin, Length: 1309, dtype: object

In [ ]:
df.isna().sum()

pclass          0
survived        0
name            0
sex             0
age           263
sibsp           0
parch           0
ticket          0
fare            1
cabin        1014
embarked        2
boat          823
body         1188
home.dest     564
dtype: int64

In [ ]:
df = df.loc[df.embarked.notna(), ['pclass', 'survived', 'sex', 'embarked']]

In [ ]:
df.isna().sum()

pclass      0
survived    0
sex         0
embarked    0
dtype: int64

In [ ]:
df.shape

(1307, 4)

In [ ]:
df.head()

,pclass,survived,sex,embarked
0,1,1,female,S
1,1,1,male,S
2,1,0,female,S
3,1,0,male,S
4,1,0,female,S


In [ ]:
df.sample(10)

,pclass,survived,sex,embarked
925,3,0,male,S
210,1,0,male,S
1106,3,0,female,S
62,1,0,male,S
124,1,1,female,C
796,3,0,male,S
169,1,0,female,C
615,3,0,male,S
598,2,1,female,S
1264,3,0,male,S


In [ ]:
df.survived.value_counts()

0    809
1    498
Name: survived, dtype: int64

In [ ]:
df.survived.value_counts(normalize=True)

0    0.618975
1    0.381025
Name: survived, dtype: float64

dica:  
Qual a accurácia mínima aceitável?  
Em casos de classificação binária como este, seria 61,89% (predição da classe predominante: '0' = Não sobreviveu!)

## Cross-validation - validação cruzada
<img src='cross-validation.png'>  
Consiste em dividir os dados em partes, exemplo k-fold = 5, dividir em 5 partes e treinar o modelo 5 vexes.  
Cada treinamento uma parte dos dados é separada para servir de teste (avaliação do modelo).

### sem cross-validation

In [ ]:
X = df.loc[:, ['pclass']]
y = df.survived

In [ ]:
X

,pclass
0,1
1,1
2,1
3,1
4,1
...,...
1304,3
1305,3
1306,3
1307,3


In [ ]:
from sklearn.linear_model import LogisticRegression

In [ ]:
reg_log = LogisticRegression(solver='lbfgs')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, 
                                                    random_state=40)

In [ ]:
reg_log.fit(X_train,y_train)

LogisticRegression()

In [ ]:
reg_log.score(X_test,y_test)

0.7633587786259542

### com cross-validation

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

In [ ]:
#df = df.sample(len(df))

In [ ]:
X = df.loc[:, ['pclass']]
y = df.survived

In [ ]:
# Havendo muitos dados, separe um conjunto de teste que você só poderá utilizar uma única vez ao final do se
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=45)

In [ ]:
reg_log = LogisticRegression(solver='lbfgs')

In [ ]:
cv_result = cross_val_score(reg_log, X_train, y_train, cv=3, scoring='accuracy')

cv_result.mean()

0.6736982511609524

In [ ]:
cv_result

array([0.65902579, 0.6954023 , 0.66666667])

In [ ]:
reg_log.fit(X_train, y_train)

LogisticRegression()

In [ ]:
reg_log.score(X_test, y_test)

0.6870229007633588

In [ ]:
cv_result.std()

0.015660885913980925

In [ ]:
cv_result = cross_val_score(reg_log, X, y, cv=10, scoring='accuracy', )

cv_result.mean()

0.6762301820317088

In [ ]:
cv_result

array([0.38167939, 0.67938931, 1.        , 0.98473282, 0.61832061,
       0.61832061, 0.61832061, 0.61538462, 0.62307692, 0.62307692])

In [ ]:
cv_result.std()

0.17498119933511946

In [ ]:
from sklearn.model_selection import KFold

In [ ]:
# Forma de visualizar quais dados entram em quais dobras do CV em treino ou teste
cv2 = KFold(2, shuffle=True, random_state=42)

In [ ]:
[k for k in cv10.split(X,y)]

NameError: ignored

In [ ]:
# Façamos um shuffle manual
df_shuffle = df.sample(frac=1).reset_index()
df_shuffle

In [ ]:
X = df_shuffle.loc[:, ['pclass']]
y = df_shuffle.survived

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=45)

In [ ]:
reg_log = LogisticRegression(solver='lbfgs')

In [ ]:
cv_result = cross_val_score(reg_log, X_train, y_train, cv=10, scoring='accuracy')

cv_result

In [ ]:
print(f'média = {cv_result.mean()}')
print(f'dp = {cv_result.std()}')

In [ ]:
cv_result = cross_val_score(reg_log, X_train, y_train, cv=5, scoring='accuracy')

cv_result

In [ ]:
print(f'média = {cv_result.mean()}')
print(f'dp = {cv_result.std()}')

In [ ]:
cv_result = cross_val_score(reg_log, X_train, y_train, cv=2, scoring='accuracy')

cv_result

In [ ]:
print(f'média = {cv_result.mean()}')
print(f'dp = {cv_result.std()}')

## Encoding variáveis (features) categoricas :
Se as variáveis não possuem uma ordem específica, geralmente a melhor opção é one hot-encoding (dummy encoding)

In [ ]:
from sklearn.preprocessing import  OneHotEncoder
hot_enc = OneHotEncoder(sparse=False)

In [ ]:
df.head()

In [ ]:
df.tail()

### Codificando só a variável 'embarked' para ver como fica

In [ ]:
['embarked_C', 'embarked_Q', 'embarked_S']

In [ ]:
df[['embarked_C', 'embarked_Q', 'embarked_S']] = hot_enc.fit_transform(df[['embarked']])

In [ ]:
df.sample(5)

In [ ]:
hot_enc.categories_

### Codificando todas as variáveis: 'sex' e 'embarked' 

In [ ]:
X = df[['pclass','sex','embarked']]

In [ ]:
X.head()

In [ ]:
X.sample(10)

In [ ]:
from sklearn.compose import make_column_transformer

In [ ]:
col_transf = make_column_transformer((OneHotEncoder(), ['sex','embarked']),
                                     remainder='passthrough', )

In [ ]:
col_transf.get_feature_names()

In [ ]:
df_transform = pd.DataFrame(col_transf.fit_transform(X), columns=col_transf.get_feature_names())

In [ ]:
df_transform

## Pipeline com codificação categóricas e cross-validation

In [ ]:
from sklearn.pipeline import make_pipeline

In [ ]:
X = df[['pclass','sex','embarked']]
y = df['survived']

In [ ]:
col_transf = make_column_transformer((OneHotEncoder(), ['sex','embarked']),
                                     remainder='passthrough')

In [ ]:
reg_log = LogisticRegression(solver='lbfgs')

In [ ]:
pipe = make_pipeline(col_transf, reg_log)

In [ ]:
pipe

In [ ]:
cross_val_score(pipe, X, y, cv=10, scoring='accuracy').mean()

In [ ]:
novo_X = X.sample(5, random_state=123)
novo_X

In [ ]:
pipe.fit(X,y)

In [ ]:
pipe.predict(novo_X)

In [ ]:
y.sample(5, random_state=123)

### pipeline tem algumas vantagens:

1- Seu arquivo de treinamento permanece o mesmo e não vai crescer por causa do one-hot encoding.  
2- Na predição de novos dados, não é necessário fazer pandas dummies no novo arquivo. Também evita eventuais problemas caso os novos dados não tenham todas as categorias que existem nos dados de treinamento. As dimensões do novo dataset será diferente e vai dar erro.  
3- É possível fazer grid search para os parâmetros de pré-processamento e os parâmetros do modelo.  
